In [45]:
import tensorflow as tf
from tensorflow.keras import layers
# -----------------------------
class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(max_len, d_model)

    def get_angles(self, pos, i, d_model):
        pos = tf.cast(pos, tf.float32)
        i = tf.cast(i, tf.float32)
        angle_rates = 1 / tf.pow(10000., (2 * (i//2)) / tf.cast(d_model, tf.float32))
        return pos * angle_rates

    def positional_encoding(self, max_len, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(max_len)[:, tf.newaxis],
            i=tf.range(d_model)[tf.newaxis, :],
            d_model=d_model
        )

        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]



        return tf.cast(pos_encoding, tf.float32)



    def call(self, x):
        return x + tf.cast(self.pos_encoding[:, :tf.shape(x)[1], :], x.dtype)

In [41]:
# -----------------------------
# Transformer Encoder Block
# -----------------------------
def transformer_encoder_block(embed_dim, num_heads, ff_dim, dropout=0.1):
    inputs = layers.Input(shape=(None, embed_dim))

    # QUESTION : Apply Layer Norm + MultiHead Attention
    x = layers.LayerNormalization()(inputs) #normalization
    attention_output = layers.MultiHeadAttention(

        num_heads=num_heads,
        key_dim=embed_dim



    )(x, x)
    x = layers.Add()([inputs, attention_output])  # Residual

    # QUESTION : Apply Feed Forward Network with Approrpriate Activation fxn
    ff = layers.LayerNormalization()(x)
    ff = ff = layers.Dense(ff_dim, activation="relu")(ff)
    ff = layers.Dense(embed_dim)(ff)
    # put your answer code block here"

    x = layers.Add()([x, ff])  # Residual

    return Model(inputs, x)


In [42]:

# -----------------------------
# Build the Transformer Model
# -----------------------------
def build_transformer(
        max_len=100,
        vocab_size=10000,
        embed_dim=64,
        num_heads=4,
        ff_dim=128,
        num_layers=3
    ):
    inputs = layers.Input(shape=(None,), dtype=tf.int32)

    # QUESTION : Apply Token Embedding and Positional Encoding

    x = layers.Embedding(vocab_size, embed_dim)(inputs)#embedding
    x = PositionalEncoding(max_len, embed_dim)(x) #positional encoding

    # 3 Transformer Encoder Blocks
    for _ in range(num_layers):
        x = transformer_encoder_block(embed_dim, num_heads, ff_dim)(x)

    # Classification head (example)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="Transformer_3Layer")
    return model

In [46]:
# -----------------------------
# Instantiate + Compile Model
# -----------------------------
model = build_transformer()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), #fill appropriate learning rate
    loss="binary_crossentropy", #fill appropriate loss fxn
    metrics=["accuracy"]
)

model.summary()

Model: "Transformer_3Layer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_35 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_15 (Embedding)        │ (None, None, 64)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding_15          │ (None, None, 64)       │             0 │
│ (PositionalEncoding)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_6 (Functional)       │ (None, None, 64)       │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_7 (Functional)       │ (None, None, 64)       │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_8 (Functional)       │ (None, None, 64)       │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 893,825 (3.41 MB)

 Trainable params: 893,825 (3.41 MB)

 Non-trainable params: 0 (0.00 B)